In [1]:
import numpy as np
import pandas as pd
import streamlit as st
import csv
from collections import defaultdict
from numpy.random import default_rng as rng

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
from os import path
from matplotlib.pyplot import figure
if 'Sarabun' not in [x.name for x in matplotlib.font_manager.fontManager.ttflist]:
    matplotlib.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
matplotlib.rc('font', family='Sarabun')

from rsi_modules import *

mm3_url         = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=1475905714"
publication_url = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=2125519370"
theses_url      = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=609352385"

F = pd.read_csv(mm3_url)
publication = pd.read_csv(publication_url).fillna("")
theses = pd.read_csv(theses_url).fillna("")
theses = theses.drop_duplicates(subset=['รหัส (นักศึกษา)'], keep='last')

In [10]:
start_date = '12/12/2025'
end_date   = '12/30/2025'

f = F[F['Timestamp'].between(start_date, end_date)]
f = f.drop_duplicates(subset=['รหัสวิชา'], keep='last')
units = summarize_course_units(f)
lec = courses(f)

0 รมคพ 707 
1 รมคพ 726
2 รมกพ 506 
3 รมกพ 508
4 รมคพ 505
5 รมคพ 506 
6 รมคพ 512 
7 รมคพ 513 
8 รมกพ 509
9 รมหศ 374
10 รมคพ 514
11 รมกพ 638
12 รมคพ 521 
13 รมหศ 376
14 รมหศ 491
15 รมหศ 494
16 ีรมหศ 447
17 รมหศ 478
18 รมศษ ๑๑๑


KeyError: 'data'

In [11]:
data = defaultdict(lambda:defaultdict(list))
lec   = defaultdict(lambda:defaultdict(float))
# ดึงทีละวิชา
for p,ci in enumerate(f["รหัสวิชา"]):
    print(p, ci)
    ci = thai_digit_to_arabic_digit(ci)

    fdi = extract_non_empty_cells(f,ci)
    # ดึงชั่วโมงบรรยาย
    hr_lecture = int(fdi[fdi['items']=="จำนวน หน่วยกิต (ทฤษฎี-ปฏิบัติ-เรียนรู้ด้วยตนเอง) [ทฤษฎี]"]['data'].to_numpy()[0])
    # ดึงผู้สอนหลัก
    fdii = fdi.loc[(fdi['data'] == 'ผู้สอนหลัก') | (fdi['data'] == 'ผู้สอนหลัก, ผู้ปฏิบัติ')].reset_index(drop=True)
    for i in range(1,17):
        J = [j for j in fdii['items'] if f'(คาบที่ {i})' in j]
        if len(J) > 0:
            data[i][ci] = hr_lecture/len(J)
            for jj in J:
                lec[ci][jj.split('[')[1].split(']')[0]]+=hr_lecture/len(J)
        else:
            data[i][ci] = 0

0 รมคพ 707 
1 รมคพ 726
2 รมกพ 506 
3 รมกพ 508
4 รมคพ 505
5 รมคพ 506 
6 รมคพ 512 
7 รมคพ 513 
8 รมกพ 509
9 รมหศ 374
10 รมคพ 514
11 รมกพ 638
12 รมคพ 521 
13 รมหศ 376
14 รมหศ 491
15 รมหศ 494
16 ีรมหศ 447
17 รมหศ 478
18 รมศษ ๑๑๑


KeyError: 'data'

In [13]:
ci

'รมศษ 111'

In [12]:
pd.DataFrame(f.loc[f["รหัสวิชา"]==ci].T.reset_index())

,index
0,Timestamp
1,Email Address
2,อาจารย์ผู้รับผิดชอบรายวิชา
3,ชื่ออาจารย์พิเศษ (นอกคณะแพทยศาสตร์โรงพยาบาลราม...
4,ชื่ออาจารย์พิเศษ (ภายนอก ม.มหิดล)
...,...
528,อาจารย์ผู้สอน/ผู้ปฏิบัติ (คาบที่ 15) [อาจารย์พ...
529,อาจารย์ผู้สอน/ผู้ปฏิบัติ (คาบที่ 16) [อาจารย์พ...
530,ชื่ออาจารย์พิเศษ (ในคณะแพทยศาสตร์โรงพยาบาลรามา...
531,ปีการศึกษา (เทอม) [2571]


In [7]:
f.head(3)

,Timestamp,Email Address,อาจารย์ผู้รับผิดชอบรายวิชา,ชื่ออาจารย์พิเศษ (นอกคณะแพทยศาสตร์โรงพยาบาลรามาธิบดี),ชื่ออาจารย์พิเศษ (ภายนอก ม.มหิดล),ชื่อวิชา,รหัสวิชา,เป้าหมายรายวิชา,คำอธิบายรายวิชา,วัตถุประสงค์รายวิชา,...,อาจารย์ผู้สอน/ผู้ปฏิบัติ (คาบที่ 10) [อาจารย์พิเศษ (ภายในคณะแพทย์ฯรามา ม.มหิดล)],อาจารย์ผู้สอน/ผู้ปฏิบัติ (คาบที่ 11) [อาจารย์พิเศษ (ภายในคณะแพทย์ฯรามา ม.มหิดล)],อาจารย์ผู้สอน/ผู้ปฏิบัติ (คาบที่ 12) [อาจารย์พิเศษ (ภายในคณะแพทย์ฯรามา ม.มหิดล)],อาจารย์ผู้สอน/ผู้ปฏิบัติ (คาบที่ 13) [อาจารย์พิเศษ (ภายในคณะแพทย์ฯรามา ม.มหิดล)],อาจารย์ผู้สอน/ผู้ปฏิบัติ (คาบที่ 14) [อาจารย์พิเศษ (ภายในคณะแพทย์ฯรามา ม.มหิดล)],อาจารย์ผู้สอน/ผู้ปฏิบัติ (คาบที่ 15) [อาจารย์พิเศษ (ภายในคณะแพทย์ฯรามา ม.มหิดล)],อาจารย์ผู้สอน/ผู้ปฏิบัติ (คาบที่ 16) [อาจารย์พิเศษ (ภายในคณะแพทย์ฯรามา ม.มหิดล)],ชื่ออาจารย์พิเศษ (ในคณะแพทยศาสตร์โรงพยาบาลรามาธิบดี),ปีการศึกษา (เทอม) [2571],ปีการศึกษา (เทอม) [2572]
51,12/12/2025 10:59:21,nattawan.fun@mahidol.ac.th,ศ.ดร.ทวี เชื้อสุวรรณทวี,NaN,NaN,สัมมนาเชิงบูรณาการในการพัฒนาคุณภาพชีวิตคนพิการ,รมคพ 707,พัฒนาผู้เรียนให้สามารถค้นหาและพัฒนาประเด็นเพื่...,การค้นหา และการพัฒนาประเด็นสำคัญผ่านกระบวนการว...,1. สามารถค้นหาและพัฒนาประเด็นปัญหาและความจำเป็...,...,NaN,ผู้สอนหลัก,NaN,NaN,NaN,NaN,NaN,อาจารย์ ดร.แพทย์หญิงอัญญพร สุทัศน์วรวุฒิ สึจิยะ,NaN,NaN
52,12/12/2025 11:07:48,nattawan.fun@mahidol.ac.th,ดร.รติรส จันทรสมดี,ผู้ช่วยศาสตราจารย์ ดร.กภ.วรินทร์ กฤตยาเกียรณ,NaN,ความพิการและผู้สูงอายุ,รมคพ 726,ปัจจุบันประเทศไทยเข้าสู่สังคมผู้สูงอายุ ซึ่งไม...,ทฤษฎี แนวปฏิบัติที่เป็นปัจจุบันที่เกี่ยวข้องสั...,1. อธิบาย ทฤษฎี และแนวปฏิบัติที่ทันสมัย เป็นปั...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,12/12/2025 11:42:08,natchanan.vat@mahidol.ac.th,ดร.รุจิรา สงขาว,NaN,NaN,หลักการศึกษาสำหรับบุคคลที่มีความต้องการพิเศษ,รมกพ 506,1.1\tนักศึกษาอธิบายความหมายและเลือกใช้คำนิยามศ...,คำนิยามศัพท์ของบุคคลที่มีความต้องการพิเศษ วิวั...,2.1 เพื่อให้นักศึกษามีความรู้และเข้าใจความหมาย...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1. ผู้ช่วยศาสตราจารย์ ดร.ธีรศักดิ์ ศรีสุรกุล แ...,NaN,NaN


In [2]:
st.title('ภาระงานของคณาจารย์ในสถาบันราชสุดาปีการศึกษา 2568')

st.subheader("1. ชั่วโมงสอนและจำนวนวิชาที่รับผิดชอบ")

academic_year = st.radio(
    "เลือกเทอมและปีการศึกษา",
    ["2568/1", "2568/2"],
)

if academic_year == "2568/1":
    start_date = '8/5/2025'
    end_date   = '9/19/2025'
elif academic_year == "2568/2":
    start_date = '12/12/2025'
    end_date   = '12/30/2025'

f = F[F['Timestamp'].between(start_date, end_date)]
t1,t2 = report_mm3(f)

st.subheader("2. ผลงานตีพิมพ์ทางวิชาการในแต่ละปี")
publication,df = get_publication(publication)
df = df.reset_index().rename(columns={'index':'ปี'})
st.write(df)

st.subheader("3. ภาระงานควบคุมวิทยานิพนธ์")
B = pd.DataFrame(get_thesis_workload_new(theses)).T
st.write(B)

st.subheader("4. ภาพรวมความก้าวหน้าวิทยานิพนธ์ของนักศึกษา")
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)
st.write(A)
st.write("หมายเหตุ:")
st.write("MA  = หส.ปริญญาโท สาขาคุณภาพชชีวิตคนพิการ")
st.write("MEd = หส.ปริญญาโท สาขาการศึกษาพิเศษ")
st.write("PhD = หส.ปรัชญาดุษฎีบัณฑิต สาขาคุณภาพชีวิตคนพิการ")

2025-12-24 13:56:05.508 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.340 
  command:

    streamlit run c:\Users\User\miniconda3\envs\rsi_dashboard\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-24 13:56:06.341 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.342 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.343 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.344 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.344 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-24 13:56:06.345 Thre

In [2]:
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)

In [3]:
st.write(A)

2025-12-17 17:05:31.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 17:05:31.468 
  command:

    streamlit run c:\Users\User\miniconda3\envs\rsi_dashboard\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-17 17:05:31.468 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-17 17:05:31.470 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
